In [2]:
import cv2
import os
import numpy as np
import glob

In [3]:
pattern_size = (11, 7)
square_size = 70

object_points = []
image_points_left = []
image_points_right = []

objp = np.zeros((np.prod(pattern_size), 3), dtype=np.float32)
objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
objp *= square_size

left_images = sorted(glob.glob('./image/left*'))
right_images = sorted(glob.glob('./image/right*'))
for left, right in zip(left_images, right_images):
    left_img = cv2.imread(left)
    right_img = cv2.imread(right)
    
    left_img = cv2.resize(left_img, (640, 480))
    right_img = cv2.resize(right_img, (640, 480))

    # Convert the images to grayscale
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners in the images
    ret_left, corners_left = cv2.findChessboardCorners(left_gray, pattern_size, None)
    ret_right, corners_right = cv2.findChessboardCorners(right_gray, pattern_size, None)

    # If the corners are found, add the points to the arrays
    if ret_left and ret_right:
        object_points.append(objp)
        image_points_left.append(corners_left)
        image_points_right.append(corners_right)

In [12]:
intrinsic = np.load("./ocam_distortion.npy", allow_pickle=True).item()

resolution=1280
K1 = np.array(intrinsic[str(resolution)]["left"]["K"].copy())
K2 = np.array(intrinsic[str(resolution)]["right"]["K"].copy())
D1 = np.array(intrinsic[str(resolution)]["left"]["D"].copy())
D2 = np.array(intrinsic[str(resolution)]["left"]["D"].copy())

In [15]:
intrinsic

{'1280': {'left': {'K': [[3019.4448329690313, 0.0, 611.4780923143829],
    [0.0, 3107.5377365583054, 404.24054971054835],
    [0.0, 0.0, 1.0]],
   'D': [[-2.709297813440373,
     15.071314707355377,
     -0.005124306301324113,
     0.10191728120455235,
     -112.61281290386643]]},
  'right': {'K': [[1134.0866083699168, 0.0, 751.5573334393581],
    [0.0, 1119.5327572461763, 384.68345458952984],
    [0.0, 0.0, 1.0]],
   'D': [[-0.3619215762150339,
     0.187400183722031,
     0.0007350606960144577,
     0.0029733929377228512,
     -0.06542037878829893]]}}}

In [13]:
image_size = left_gray.shape[::-1]

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
flags = cv2.CALIB_FIX_INTRINSIC
retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = cv2.stereoCalibrate(
    objectPoints=object_points, imagePoints1=image_points_left, imagePoints2=image_points_right, 
    cameraMatrix1=K1, distCoeffs1=None, cameraMatrix2=K2, distCoeffs2=None, 
    imageSize=image_size, R=None, T=None, criteria=criteria, flags=flags)

R1, R2, P1, P2, Q, validPixROI1, validPixROI2 = cv2.stereoRectify(
    cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, image_size, R, T, alpha=1)

In [11]:
R, T

(array([[ 0.92725266, -0.12791853, -0.35190816],
        [ 0.07075171,  0.98276087, -0.17080768],
        [ 0.36769104,  0.13348377,  0.92031809]]),
 array([[-1018.94474716],
        [  416.22900863],
        [ -143.53173021]]))

In [14]:
R, T

(array([[ 0.88990845,  0.00372572, -0.45612397],
        [ 0.04296844,  0.99483534,  0.09195846],
        [ 0.45411085, -0.10143354,  0.88515229]]),
 array([[ 1205.82849883],
        [ -607.86808246],
        [-3890.71891541]]))

In [6]:
radian = 20 * np.pi / 180

cos_v = np.cos(radian)
sin_v = np.sin(radian)

R1 = [[cos_v, 0.0, -sin_v], [0.0, 1.0, 0.0], [sin_v, 0.0, cos_v]]
t1 = np.array([[-0.08],[0],[0]])

R2 = [[cos_v, 0.0, sin_v], [0.0, 1.0, 0.0], [-sin_v, 0.0, cos_v]]
t2 = np.array([[0.08],[0],[0]])

R = np.matmul(np.linalg.inv(R2), R1)
T = np.matmul(np.linalg.inv(R2), (t1 - t2))

In [7]:
R, T

(array([[ 0.76604444,  0.        , -0.64278761],
        [ 0.        ,  1.        ,  0.        ],
        [ 0.64278761,  0.        ,  0.76604444]]),
 array([[-0.15035082],
        [ 0.        ],
        [-0.05472322]]))